<a href="https://colab.research.google.com/github/anton-akulenko/DA-test/blob/main/bquery_level2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# instal *dependencies*

In [ ]:
# pip install --upgrade google-cloud-bigquery

# Imports

In [5]:
import os
import json
from google.cloud import bigquery
from google.oauth2 import service_account
import time

import pandas as pd
import gspread
from google.colab import auth
from google.oauth2.service_account import Credentials
from concurrent.futures import ThreadPoolExecutor

In [2]:
from google.colab import files
uploaded = files.upload()

Saving arctic-math-419406-a1f56944d811.json to arctic-math-419406-a1f56944d811.json


In [6]:
auth.authenticate_user()
credentials = Credentials.from_service_account_file('/content/arctic-math-419406-a1f56944d811.json')
client = bigquery.Client(credentials=credentials, project=credentials.project_id)


In [7]:
queries = []

queries += [f"SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707{str(day).zfill(2)}`" for day in range(1, 31)]
queries += [f"SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201705{str(day).zfill(2)}`" for day in range(1, 31)]


In [8]:
len(queries)

60

In [9]:
def fetch_data(query):
    start_time = time.time()
    query_job = client.query(query)
    df = query_job.to_dataframe()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Query '{query}' completed in {elapsed_time} seconds")
    return df


def plain_df(df):
    df_expanded = pd.concat([df.drop(['totals', 'trafficSource', 'geoNetwork', 'device', 'customDimensions', 'visitorId', 'hits'], axis=1),
                         df['totals'].apply(pd.Series).add_prefix('total_'),
                         df['device'].apply(pd.Series).add_prefix('device_'),
                        #  df['hits'].apply(pd.Series).add_prefix('hits_'),
                         df['geoNetwork'].apply(pd.Series).add_prefix('geoNetwork_')], axis=1)
    return df_expanded


def combine_dataframes(dfs):
    return pd.concat(dfs, ignore_index=True)


def group_and_aggregate(df, col_name):
    aggregated = df.groupby(col_name).agg({'device_browser': ['max', 'min'],
                                            'total_pageviews': ['median'],
                                            'total_visits': ['count'],
                                            'device_deviceCategory': ['max', 'min'],
                                            'geoNetwork_country': 'first'
                                        }).reset_index()
    aggregated.columns = [col + '_' + func if col != col_name else col for col, func in aggregated.columns]

    return aggregated



In [ ]:
total_start_time = time.time()

with ThreadPoolExecutor(max_workers=8) as executor:
    dfs = list(executor.map(fetch_data, queries))

combined_df = combine_dataframes(dfs)

df_no_dict = plain_df(combined_df)
total_end_time = time.time()
total_elapsed_time = total_end_time - total_start_time
print(f"Total time: {total_elapsed_time} seconds")


In [26]:
total_start_time = time.time()
column_to_group = ['geoNetwork_subContinent', 'date', 'visitStartTime', 'geoNetwork_country']
grouped_df1 = group_and_aggregate(df_no_dict, column_to_group[0])
grouped_df2 = group_and_aggregate(df_no_dict, column_to_group[1])
grouped_df3 = group_and_aggregate(df_no_dict, column_to_group[2])
grouped_df4 = group_and_aggregate(df_no_dict, column_to_group[3])


total_end_time = time.time()
total_elapsed_time = total_end_time - total_start_time
print(f"Total time: {total_elapsed_time} seconds")

Total time: 52.36764669418335 seconds


In [ ]:
combined_df.head()

In [ ]:
df_no_dict.head(10)

In [ ]:
grouped_df1

In [ ]:
grouped_df2

In [ ]:
grouped_df3

In [16]:
combined_df.shape

(132271, 16)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

/content/drive/MyDrive/tests_colab/DA/

In [27]:
def write_to_excel(df, sheet_name, writer):
    start_time = time.time()
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"\nSheet '{sheet_name}' finished in {elapsed_time} seconds")

total_start_time = time.time()

with pd.ExcelWriter('/content/drive/MyDrive/tests_colab/DA/data.xlsx') as writer:
    with ThreadPoolExecutor(max_workers=8) as executor:
        # executor.submit(write_to_excel, combined_df, 'Combined Data', writer)
        executor.submit(write_to_excel, grouped_df1, 'Grouped Data 1', writer)
        executor.submit(write_to_excel, grouped_df2, 'Grouped Data 2', writer)
        executor.submit(write_to_excel, grouped_df3, 'Grouped Data 3', writer)
        executor.submit(write_to_excel, grouped_df4, 'Grouped Data 4', writer)

total_end_time = time.time()
total_elapsed_time = total_end_time - total_start_time
print(f"Total time: {total_elapsed_time} seconds")



Sheet 'Grouped Data 1' finished in 0.003812551498413086 seconds

Sheet 'Grouped Data 2' finished in 0.014716386795043945 seconds

Sheet 'Grouped Data 4' finished in 0.029042959213256836 seconds

Sheet 'Grouped Data 3' finished in 12.534766435623169 seconds
Total time: 27.14944052696228 seconds
